In [ ]:
%load_ext autoreload
%autoreload 2

from torchsummary import summary

from ppg_project.dl_training_pipeline import DLTrainingPipeline
from ppg_project.dtypes import DLTrainingPipelineConfig
from ppg_project.ppg_models import SpectralCNN
from ppg_project import utils

<h1> Define DL model <h1>

In [ ]:
model = SpectralCNN(
    hidden_dim=4,
    hidden_size=256,
    input_data=(127, 100),
    max_pool=True)

summary(model, input_size=(1, 127, 100))

<h1> Initialize DL pipeline and load data <h1>

In [ ]:
config = DLTrainingPipelineConfig(
    batch_size=64,
    n_epochs=30,
    appply_segmentation=True,
    early_stopping_tolerance=3,
    learning_rate=1e-4,
    dl_features="wavelet"
    )

pipe = DLTrainingPipeline(config,model)
data = pipe.train
target = pipe.train_label

<h1> Apply window segmentation <h1>

In [ ]:
if config.appply_segmentation:
    data,target = utils.segment_data_in_window(
                    data,
                    target,
                    nsec = config.segment_window_sec, 
                    sampling_rate = config.sampling_rate)

In [ ]:
data.drop(columns=["sample_index","window_index"],inplace=True)
data

<h1> Take first window only <h1>

In [ ]:
list_feature_col = [c for c in data.columns if c.startswith("feature")]
data = data.drop(columns=list_feature_col).values[:60000]
target = target[:60000]
print(data.shape,target.shape)

<h1> Extract features (wavelet or spectrogram) <h1>

In [ ]:
data_processed = pipe.process_data(data,target)

In [ ]:
data_processed

<h1> Launch training <h1>

In [ ]:
pipe.run_holdhout(data_processed)